# 데이터 로드

In [108]:
import requests
from bs4 import BeautifulSoup
import json
import time
from dotenv import load_dotenv
import os
from datetime import datetime
import pandas as pd

In [109]:
def get_today_weather_data():
    """네이버에서 서울 오늘 날씨 크롤링"""
    
    url = "https://search.naver.com/search.naver"
    params = {'query': '서울 날씨'}
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 날씨 데이터 추출
        weather_data = {
            '날짜': datetime.now().strftime("%Y년 %m월 %d일"),
            '지역': '서울'
        }
        
        # 현재 온도
        temp_element = soup.select_one('.temperature_text')
        if temp_element:
            weather_data['현재온도'] = temp_element.get_text().strip()
        else:
            weather_data['현재온도'] = "정보 없음"
        
        # 날씨 상태
        weather_element = soup.select_one('.summary_list .sort')
        if weather_element:
            weather_data['날씨상태'] = weather_element.get_text().strip()
        else:
            weather_data['날씨상태'] = "정보 없음"
        
        # 날씨 상태 (비, 눈, 흐림 등)
        weather_before_slash = soup.select_one('.weather.before_slash')
        if weather_before_slash:
            weather_data['날씨상세'] = weather_before_slash.get_text().strip()
        else:
            weather_data['날씨상세'] = "정보 없음"
        
        # 미세먼지
        dust_elements = soup.select('.today_chart_list .item_today')
        for element in dust_elements:
            text = element.get_text()
            if '미세먼지' in text and '초미세먼지' not in text:
                weather_data['미세먼지'] = element.select_one('.txt').get_text().strip()
            elif '초미세먼지' in text:
                weather_data['초미세먼지'] = element.select_one('.txt').get_text().strip()
        
        # 기본값 설정
        if '미세먼지' not in weather_data:
            weather_data['미세먼지'] = "정보 없음"
        if '초미세먼지' not in weather_data:
            weather_data['초미세먼지'] = "정보 없음"
        
        print(" 날씨 크롤링 성공!")
        return weather_data
        
    except Exception as e:
        print(f" 크롤링 실패: {e}")
        return None

In [110]:
get_today_weather_data()

 날씨 크롤링 성공!


{'날짜': '2025년 07월 16일',
 '지역': '서울',
 '현재온도': '현재 온도23.0°',
 '날씨상태': '체감 25.6°',
 '날씨상세': '비',
 '미세먼지': '좋음',
 '초미세먼지': '좋음'}

In [116]:
def save_today_weather(weather_data):
    """오늘 날씨를 고정 파일명으로 업데이트"""
    
    filename = "오늘날씨.xlsx"  # 항상 같은 파일명
    
    if weather_data:
        # 새 데이터를 DataFrame으로 변환
        df = pd.DataFrame([weather_data])
        
        # 기존 파일 덮어쓰기 (매일 새로운 내용으로 교체)
        df.to_excel('today_data/오늘날씨.xlsx', index=False)
        
        print(f"오늘 날씨 업데이트 완료!")
        print(f"파일: {filename}")
        print(f"날짜: {weather_data['날짜']}")
        print(f"온도: {weather_data['현재온도']}")
        
        return filename
    else:
        print("저장할 날씨 데이터가 없습니다.")
        return None

In [117]:
save_today_weather(weather_data=get_today_weather_data())

 날씨 크롤링 성공!
오늘 날씨 업데이트 완료!
파일: 오늘날씨.xlsx
날짜: 2025년 07월 16일
온도: 현재 온도22.7°


'오늘날씨.xlsx'

In [118]:
def save_weather_with_backup(weather_data):
    """백업 포함 업데이트"""
    
    filename = "오늘날씨.xlsx"
    
    if weather_data:
        # 기존 파일이 있으면 백업
        if os.path.exists(filename):
            backup_time = datetime.now().strftime("%Y%m%d")
            backup_name = f"어제날씨_{backup_time}.xlsx"
            
            # 어제 파일을 백업으로 이름 변경
            try:
                import shutil
                shutil.copy(filename, backup_name)
                print(f"어제 날씨 백업: {backup_name}")
            except:
                pass
        
        # 오늘 날씨로 덮어쓰기
        df = pd.DataFrame([weather_data])
        df.to_excel(filename, index=False)
        
        print(f"오늘 날씨 업데이트!")
        print(f"{weather_data['날짜']} - {weather_data['현재온도']}")
        
        return filename
    
    return None


In [119]:
def update_today_weather():
    """매일 실행하는 함수"""
    
    # 날씨 크롤링 (기존 함수 사용)
    weather_data = get_today_weather_data()
    
    if weather_data:
        # 파일 업데이트
        save_today_weather(weather_data)
        
        print("\n 업데이트 완료!")
        print("매일 같은 파일이 오늘 날씨로 갱신됩니다.")
    else:
        print("날씨 크롤링 실패")

In [120]:
if __name__ == "__main__":
    update_today_weather()

 날씨 크롤링 성공!
오늘 날씨 업데이트 완료!
파일: 오늘날씨.xlsx
날짜: 2025년 07월 16일
온도: 현재 온도22.7°

 업데이트 완료!
매일 같은 파일이 오늘 날씨로 갱신됩니다.
